<p> Regression experiments on all dataset, where Environment is filtered </p>

In [1]:
import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, max_error, median_absolute_error, mean_absolute_error
from sklearn.metrics import mean_squared_error
import time


2023-11-21 09:55:25.124639: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-21 09:55:25.223798: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-21 09:55:25.223862: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-21 09:55:25.227383: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-21 09:55:25.238967: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-21 09:55:25.239582: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
df = pd.read_csv('data/dataMoney.csv')

In [3]:
df.shape

(2020, 15)

In [4]:
df.head()

,max_discharge_sum_r1,max_discharge_sum_r2_1,max_discharge_sum_r2_2,max_discharge_sum_r3,flood_sum_r1,flood_sum_r2_1,flood_sum_r2_2,flood_sum_r3,HW_inund_uncertainty,bridge_scour_uncertainty,travel_demand_uncertainty,restoration_capacity_uncertainty,dir_costs,lost_trips_costs_s_r,traffic_costs_s_r
0,443.457856,1658.236820,1658.442362,924.207396,7704.283023,30959.150563,19494.507734,18060.945333,1.237179,1.066024,1.178643,7,2.396527e+07,3.771578e+07,1.346267e+07
1,525.178784,1658.659798,1658.659798,376.518772,8042.074336,30948.775455,19495.723976,12105.892278,0.805171,0.920064,1.189389,5,2.156163e+07,3.629843e+07,1.427793e+07
2,525.178784,1658.659798,1658.659798,376.518772,8042.074336,30948.775455,19495.723976,12105.892278,0.895325,1.052411,0.782405,5,2.187945e+07,2.450664e+07,-6.527196e+06
3,528.794113,1659.966169,1660.085687,376.518762,8056.793807,30980.988568,19504.576908,12106.087505,0.748587,0.955364,1.143878,6,2.195877e+07,3.199478e+07,1.273019e+07
4,523.393056,1660.295347,1660.307923,379.856164,8055.296723,30984.114890,19505.358354,12140.315910,0.965187,0.831900,1.344699,6,2.236592e+07,3.822993e+07,2.754356e+07


In [5]:
X = df.drop(columns=['dir_costs', 'lost_trips_costs_s_r', 'traffic_costs_s_r'], axis=1)
y = df['lost_trips_costs_s_r']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
from sklearn.preprocessing import MaxAbsScaler

scaler = MaxAbsScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
from sklearn.metrics import mean_absolute_error
from numpy import sqrt

def print_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    print(f'Mean Squared Error (MSE):              {mse:.10f}')
    
    rmse = sqrt(mse)
    print(f'Root Mean Squared Error (RMSE):        {rmse:.10f}')
    
    mae = mean_absolute_error(y_true, y_pred)
    print(f'Mean Absolute Error (MAE):             {mae:.10f}')
    
    r2 = r2_score(y_true, y_pred)
    print(f'R-squared (R²):                        {r2:.10f}')
    
    mape = mean_absolute_percentage_error(y_true, y_pred)
    print(f'Mean Absolute Percentage Error (MAPE): {mape:.10f}')    
    
    me = max_error(y_true, y_pred)
    print(f'Max Error (ME):                        {me:.10f}')    
    
    med_ae = median_absolute_error(y_true, y_pred)
    print(f'Median Absolute Error (MedAE):         {med_ae:.10f}')    

<h2> Random Forest </h2>

In [9]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
start_time = time.time()
rf_model.fit(X_train, y_train)
print(f'Training time: {time.time() - start_time:.4f} seconds')

Training time: 1.8141 seconds


In [10]:
print("Random Forest: train")
y_pred = rf_model.predict(X_train)
print_metrics(y_train, y_pred)
print()
print("Random Forest: test")
y_pred = rf_model.predict(X_test)
print_metrics(y_test, y_pred)


Random Forest: train
Mean Squared Error (MSE):              48267769315385.2265625000
Root Mean Squared Error (RMSE):        6947500.9402939435
Mean Absolute Error (MAE):             4056142.3198926095
R-squared (R²):                        0.9535856212
Mean Absolute Percentage Error (MAPE): 0.0564752172
Max Error (ME):                        61045931.9575816989
Median Absolute Error (MedAE):         1993269.4134518150

Random Forest: test
Mean Squared Error (MSE):              322896779447651.6250000000
Root Mean Squared Error (RMSE):        17969328.8535674475
Mean Absolute Error (MAE):             11480547.1634653844
R-squared (R²):                        0.6314645181
Mean Absolute Percentage Error (MAPE): 0.1613646578
Max Error (ME):                        91310449.2345153093
Median Absolute Error (MedAE):         6193979.4339905977


<h2> Support vector regression </h2>

In [11]:
from sklearn.svm import SVR
svr_model = SVR(kernel='rbf', gamma='auto', coef0=0.0, epsilon=0.001, max_iter=-1)
start_time = time.time()
svr_model.fit(X_train, y_train)
print(f'Training time: {time.time() - start_time:.4f} seconds')

Training time: 0.2248 seconds


In [12]:
print("SVR: train")
y_pred = svr_model.predict(X_train)
print_metrics(y_train, y_pred)
print()
print("SVR: test")
y_pred = svr_model.predict(X_test)
print_metrics(y_test, y_pred)

SVR: train
Mean Squared Error (MSE):              1112217379770442.1250000000
Root Mean Squared Error (RMSE):        33349923.2348508127
Mean Absolute Error (MAE):             20894635.8741760962
R-squared (R²):                        -0.0695103473
Mean Absolute Percentage Error (MAPE): 0.3284625788
Max Error (ME):                        281969223.8731108904
Median Absolute Error (MedAE):         12850286.6946278363

SVR: test
Mean Squared Error (MSE):              956379168612562.5000000000
Root Mean Squared Error (RMSE):        30925380.6542872228
Mean Absolute Error (MAE):             20907983.0897524916
R-squared (R²):                        -0.0915551972
Mean Absolute Percentage Error (MAPE): 0.3378947282
Max Error (ME):                        138197626.8672106266
Median Absolute Error (MedAE):         14311533.5073020272


<h2> Ridge regression </h2>

In [13]:
from sklearn.linear_model import Ridge, Lasso

ridge_model = Ridge(alpha=1.0)  # alpha - regularization strength
start_time = time.time()
ridge_model.fit(X_train, y_train)
print(f'Training time: {time.time() - start_time:.4f} seconds')


Training time: 0.0116 seconds


In [14]:
print("Ridge regression: train")
y_pred = ridge_model.predict(X_train)
print_metrics(y_train, y_pred)
print()
print("Ridge regression: test")
y_pred = ridge_model.predict(X_test)
print_metrics(y_test, y_pred)

Ridge regression: train
Mean Squared Error (MSE):              337927999385095.7500000000
Root Mean Squared Error (RMSE):        18382818.0479788184
Mean Absolute Error (MAE):             11583291.7955560163
R-squared (R²):                        0.6750477932
Mean Absolute Percentage Error (MAPE): 0.1715434982
Max Error (ME):                        143858270.3383001685
Median Absolute Error (MedAE):         7754073.0218432695

Ridge regression: test
Mean Squared Error (MSE):              317445573579869.4375000000
Root Mean Squared Error (RMSE):        17817002.3735719770
Mean Absolute Error (MAE):             11608629.4857876897
R-squared (R²):                        0.6376862054
Mean Absolute Percentage Error (MAPE): 0.1751465790
Max Error (ME):                        85504989.9734045267
Median Absolute Error (MedAE):         7691729.5076978877


<h2> Lasso regression </h2>

In [15]:
# Ridge Regression
lasso_model = Lasso(alpha=1.0)  # You can adjust the regularization strength (alpha) as needed
start_time = time.time()
lasso_model.fit(X_train, y_train)
print(f'Training time: {time.time() - start_time:.4f} seconds')

Training time: 0.0599 seconds


In [16]:
print("Lasso regression: train")
y_pred = lasso_model.predict(X_train)
print_metrics(y_train, y_pred)
print()
print("Lasso regression: test")
y_pred = lasso_model.predict(X_test)
print_metrics(y_test, y_pred)

Lasso regression: train
Mean Squared Error (MSE):              333020975441578.4375000000
Root Mean Squared Error (RMSE):        18248862.3054035455
Mean Absolute Error (MAE):             11754593.9001194313
R-squared (R²):                        0.6797663968
Mean Absolute Percentage Error (MAPE): 0.1771191717
Max Error (ME):                        132954986.5004786849
Median Absolute Error (MedAE):         7958985.5155577157

Lasso regression: test
Mean Squared Error (MSE):              327697064553661.3125000000
Root Mean Squared Error (RMSE):        18102404.9383959286
Mean Absolute Error (MAE):             12052146.7977269944
R-squared (R²):                        0.6259857537
Mean Absolute Percentage Error (MAPE): 0.1845097547
Max Error (ME):                        83320830.9655300677
Median Absolute Error (MedAE):         8182145.6326460913


<h2> GRNN </h2>

In [17]:
import math
from sklearn.base import BaseEstimator, ClassifierMixin


class GRNN(BaseEstimator, ClassifierMixin):
    def __init__(self, name = "GRNN", sigma = 0.1):
        self.name = name
        self.sigma = 2 * np.power(sigma, 2)
        
    def predict(self, instance_X, train_X, train_y):
        gausian_distances = np.exp(-np.power(np.sqrt((np.square(train_X-instance_X).sum(axis=1))),2)\
                                   / self.sigma)
        gausian_distances_sum = gausian_distances.sum()
        if gausian_distances_sum < math.pow(10, -7): gausian_distances_sum = math.pow(10, -7)
        result = np.multiply(gausian_distances, train_y).sum() / gausian_distances_sum
        return result

In [18]:
for s in np.arange(0.1, 0.2, 0.001):
    grnn = GRNN(sigma=s)
    start_time = time.time()
    predictions = np.apply_along_axis(lambda i: grnn.predict(i, X_train, y_train), axis=1, arr=X_test)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("Sigma: " + str(s))
    print_metrics(y_test, predictions)


--- 0.1137702465057373 seconds ---
Sigma: 0.1
Mean Squared Error (MSE):              379473216720944.8125000000
Root Mean Squared Error (RMSE):        19480072.2976313606
Mean Absolute Error (MAE):             12543969.6781589612
R-squared (R²):                        0.5668914846
Mean Absolute Percentage Error (MAPE): 0.2053452208
Max Error (ME):                        96510157.4595734179
Median Absolute Error (MedAE):         8888990.7908037528
--- 0.10601329803466797 seconds ---
Sigma: 0.101
Mean Squared Error (MSE):              379907280518133.6875000000
Root Mean Squared Error (RMSE):        19491210.3400002755
Mean Absolute Error (MAE):             12571446.3332848549
R-squared (R²):                        0.5663960696
Mean Absolute Percentage Error (MAPE): 0.2062683310
Max Error (ME):                        96608262.3236446083
Median Absolute Error (MedAE):         8977918.2734873705
--- 0.11072921752929688 seconds ---
Sigma: 0.10200000000000001
Mean Squared Error (MSE):       

<h3> Stochastic Gradient Descent Regressor </h3>

In [19]:
from sklearn.linear_model import SGDRegressor
sgd_model = SGDRegressor(loss = 'squared_epsilon_insensitive', alpha=0.0001)

start_time = time.time()
sgd_model.fit(X_train, y_train)
print(f'Training time: {time.time() - start_time:.4f} seconds')

Training time: 0.0245 seconds


In [20]:
print("SGD Regressor: train")
y_pred = sgd_model.predict(X_train)
print_metrics(y_train, y_pred)
print()
print("SGD Regressor: test")
y_pred = sgd_model.predict(X_test)
print_metrics(y_test, y_pred)

SGD Regressor: train
Mean Squared Error (MSE):              347574546809830.8750000000
Root Mean Squared Error (RMSE):        18643351.2762547843
Mean Absolute Error (MAE):             12064391.6081677433
R-squared (R²):                        0.6657716549
Mean Absolute Percentage Error (MAPE): 0.1830664702
Max Error (ME):                        145452818.9827950299
Median Absolute Error (MedAE):         8339011.5785366036

SGD Regressor: test
Mean Squared Error (MSE):              316440323505250.7500000000
Root Mean Squared Error (RMSE):        17788769.5894137323
Mean Absolute Error (MAE):             11831042.6757712793
R-squared (R²):                        0.6388335390
Mean Absolute Percentage Error (MAPE): 0.1828673806
Max Error (ME):                        86339046.5070112199
Median Absolute Error (MedAE):         8305768.6651903018


<h2> ADA Boost regressor </h2>

In [21]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

adaboost_model = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4), n_estimators=300)

start_time = time.time()
adaboost_model.fit(X_train, y_train)
print(f'Training time: {time.time() - start_time:.4f} seconds')

Training time: 1.5837 seconds


In [22]:
print("ADA boost Regressor: train")
y_pred = adaboost_model.predict(X_train)
print_metrics(y_train, y_pred)
print()
print("ADA boost Regressor: test")
y_pred = adaboost_model.predict(X_test)
print_metrics(y_test, y_pred)

ADA boost Regressor: train
Mean Squared Error (MSE):              643843013123232.1250000000
Root Mean Squared Error (RMSE):        25374061.8175969645
Mean Absolute Error (MAE):             24117471.1434448846
R-squared (R²):                        0.3808793343
Mean Absolute Percentage Error (MAPE): 0.4927792407
Max Error (ME):                        41224816.3911699653
Median Absolute Error (MedAE):         24789485.3830614649

ADA boost Regressor: test
Mean Squared Error (MSE):              708385647296607.2500000000
Root Mean Squared Error (RMSE):        26615515.1612101495
Mean Absolute Error (MAE):             24691319.1150060892
R-squared (R²):                        0.1914900907
Mean Absolute Percentage Error (MAPE): 0.4926240075
Max Error (ME):                        69512914.6366032362
Median Absolute Error (MedAE):         24975810.3324383497


<h2> Multi-layer Perceptron regressor </h2>

In [23]:
from sklearn.neural_network import MLPRegressor
mlp_model = MLPRegressor(hidden_layer_sizes=(100, 40, 20), activation='relu', solver='adam', alpha=0.0001, batch_size='auto', max_iter=5000)

start_time = time.time()
mlp_model.fit(X_train, y_train)
print(f'Training time: {time.time() - start_time:.4f} seconds')

Training time: 40.9760 seconds


In [24]:
print("MLP Regressor: train")
y_pred = mlp_model.predict(X_train)
print_metrics(y_train, y_pred)
print()
print("MLP Regressor: test")
y_pred = mlp_model.predict(X_test)
print_metrics(y_test, y_pred)

MLP Regressor: train
Mean Squared Error (MSE):              349674061318709.7500000000
Root Mean Squared Error (RMSE):        18699573.8271948248
Mean Absolute Error (MAE):             12120453.0837837476
R-squared (R²):                        0.6637527577
Mean Absolute Percentage Error (MAPE): 0.1847223796
Max Error (ME):                        134486991.6777173579
Median Absolute Error (MedAE):         8231825.0095479600

MLP Regressor: test
Mean Squared Error (MSE):              331489759326637.1250000000
Root Mean Squared Error (RMSE):        18206860.2270308286
Mean Absolute Error (MAE):             12137855.7792671733
R-squared (R²):                        0.6216569939
Mean Absolute Percentage Error (MAPE): 0.1868182982
Max Error (ME):                        86724272.8301086426
Median Absolute Error (MedAE):         8600631.0255973153
